In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'

import sys
import time
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import pygeohash as pgh
import ast
import pandas as pd

def sendDataToDB(iter):
    client = MongoClient()
    db = client.asgn_3182    
    climate_date = db.climate_date
    hotspot_col = db.hotspot_col
    
    prod1_flag = False # Use to check if the stream contained prod1
    prod1_lat = None
    prod1_long = None
    prod2_data = []
    prod3_data = []
    fire_event = []
    merged_lst = []
    # First part
    # Store message into local variable
    for record in iter:
#         print(record)
        # Make data model based from part A
        key = record[0]
        dic_data = ast.literal_eval(record[1])
        if key == "producer1":
            prod1_flag = True
            prod1_data = dic_data # data is a dictionary
            prod1_lat = prod1_data['latitude']
            prod1_long = prod1_data['longitude']
            keys_to_extract = ['GHI_w/m2', 'date', 'relative_humidity', 'max_wind_speed', 'precipitation ', 'windspeed_knots', 'air_temperature_celcius']
            climate_data = {key: prod1_data[key] for key in keys_to_extract}
            climate_data['date'] = pd.to_datetime(climate_data['date'])
            climate_data['date'] = climate_data['date'].strftime("%d-%m-%Y")
        elif key == 'producer2':
            prod2_data.append(dic_data)
        elif key == 'producer3':
            prod3_data.append(dic_data)
    
    # Second part
    # Data processing    
    # If no climate data (prod1), terminate
    if not prod1_flag:
        client.close()
        return
    
    # If there is/are no data from hotspot (aqua or terra), store 
    # climate data into db
    if prod1_flag and prod2_data == [] and prod3_data == []:
        # Just store climate into db
        print("Insert climate data")
        climate_date.insert_one(climate_data)
        client.close()
        return

    # 1. Compare climate and hotspot with precision 3
    # If not close, ignore hotspot
    # Check between prod1 and prod2
    if prod1_flag and prod2_data != []:
        for record in prod2_data:  #loc is a tuple: (lat, long)
            lat = record['latitude']
            long = record['longitude']
            h1 = pgh.encode(prod1_lat, prod1_long, precision=3)
            h2 = pgh.encode(lat, long, precision=3)
            if h1 == h2: # If climate and hotspot is close
                print("Prod2 fire")
                fire_event.append(record)
                
    # Check between prod1 and prod3
    if prod1_flag and prod3_data != []:
        for record in prod3_data:
            lat = record['latitude']
            long = record['longitude']
            h1 = pgh.encode(prod1_lat, prod1_long, precision=3)
            h2 = pgh.encode(lat, long, precision=3)
            if h1 == h2: # If climate and hotspot is close
                print("Prod3 fire")
                fire_event.append(record)
    
    # If after precision3 checking, no close hotspots, store climate into mongoDB
    # and return from function
    if fire_event == []:
        print("Insert climate data")
        climate_date.insert_one(climate_data)
        client.close()
        return 
    
    # Check with hotspot data that are close as fire event
    # Convert all data of filtered_prod2_data and filtered_prod3_data into pandas df
    if fire_event != []:
        df3 = pd.DataFrame.from_dict(fire_event) # Dataframe for producer 2
        df3['geohash'] = df3.apply(lambda col: pgh.encode(col.latitude, col.longitude, precision=5), axis = 1)
        df3['date'] = pd.to_datetime(df3.date) # Convert 'date' from type string to type datetime
        df3['date'] = df3['date'].dt.strftime("%d-%m-%Y") # Convert modified format to default format due to pandas default config
        df4 = df3.groupby(['geohash']).agg(['mean'])
        for index, row in df4.iterrows(): # If merged dataframe with aggregate count is not empty,
            print("Merge prec5")
            merged_data = {}
            merged_data['latitude'] = float(row['latitude']['mean'])
            merged_data['longitude'] = float(row['longitude']['mean'])
            merged_data['confidence'] = int(row['confidence']['mean'])
            merged_data['surface_temperature_celcius'] = int(row['surface_temperature_celcius']['mean'])
            merged_data['date'] = str(climate_data['date'])  # Take climate date
            merged_data['datetime'] = str(df3['datetime'].max())  # Take latest datetime
            merged_lst.append(merged_data)
            
#     3. Decide the fire event is 'natural' or 'other' based on 
#        air temp>20 and GHI > 180. append a new field to the hotspot
    if merged_lst != []:
        # Insert climate data
        print("Insert climate data")
        climate_date.insert_one(climate_data)
        # Check for natural or other
        for record in merged_lst:
            if climate_data['air_temperature_celcius'] > 20 and climate_data['GHI_w/m2'] > 180:
                record['cause_of_fire'] = 'natural'
            else:
                record['cause_of_fire'] = 'other'
            keys_to_extract = ['datetime', 'surface_temperature_celcius', 'confidence', 'date', 'cause_of_fire']
            reports = [{key: record[key] for key in keys_to_extract}]
            print("Insert hotspot data")
            hotspot_col.insert_one({"latitude": merged_data['latitude'],
                                    "longitude": merged_data['longitude'],
                                    "reports": reports})
    client.close()

n_secs = 10
topic1 = "asgn_producer"


conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic1], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week11-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary

lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)